## 8日目
### 便利なモナディック関数特集
* Scalaz の Monad は Applicative を継承している為、全てのモナドが Functor であることが保証される
* つまり、 map や <*> 演算子も使える

In [1]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

### join メソッド
* 実は、任意の入れ子になったモナドは平らにできる
* これはモナド特有の性質

* Scalaz では join メソッドは Bind によって導入される

```scala
trait BindOps[F[_], A] exenteds Ops[F[A]] {
    ...
    def join[B](implicit ev: A <-< F[B]): F[B] = F.bind(self)(ev(_))
    def μ[B](implicit ev: A <-< F[B]): F[B] = F.bind(self)(ev(_))
    ...
}
```

In [2]:
(Some(9.some): Option[Option[Int]]).join

res1: Option[Int] = Some(9)

In [3]:
(Some(none): Option[Option[Int]]).join

res2: Option[Int] = None

In [4]:
List(List(1, 2, 3), List(4, 5, 6)).join

res3: List[Int] = List(1, 2, 3, 4, 5, 6)

In [5]:
9.right[String].right[String].join

res4: String \/ Int = \/-(9)

In [6]:
"boom".left[Int].right[String].join

res5: String \/ Int = -\/("boom")

### filterM メソッド
* Control.Monad モジュールの filterM こそ、まさにそのための関数
* 述語は Bool の結果とするモナド値を返している

* Scalaz では filterM はいくつかの箇所で実装されている

```scala
trait ListOps[A] extends Ops[List[A]] {
    ...
    final def filterM[M[_]] : Monad](p: A => M[Boolean]): M[List[A]] = l.filterM(self)(p)
    ...
}
```

In [7]:
List(1, 2, 3) filterM { x => List(true, false) }

res6: List[List[Int]] = List(
  List(1, 2, 3),
  List(1, 2),
  List(1, 3),
  List(1),
  List(2, 3),
  List(2),
  List(3),
  List()
)

In [8]:
Vector(1, 2, 3) filterM {x => Vector(true, false) }

res7: Vector[Vector[Int]] = Vector(
  Vector(1, 2, 3),
  Vector(1, 2),
  Vector(1, 3),
  Vector(1),
  Vector(2, 3),
  Vector(2),
  Vector(3),
  Vector()
)

### foldLeftM メソッド
* foldl のモナド版が foldM

* Scalaz でこれは Foldable に foldLeftM として実装されていて foldRightM もある

In [9]:
def binSmalls(acc: Int, x: Int): Option[Int] = {
    if (x > 9) (none: Option[Int])
    else (acc + x).some
}

defined function binSmalls

In [10]:
List(2, 8, 3, 1).foldLeftM(0) {binSmalls}

res9: Option[Int] = Some(14)

In [11]:
List(2, 11, 3, 1).foldLeftM(0) {binSmalls}

res10: Option[Int] = None

### 安全な RPN 電卓を作ろう
* 第１０章で逆ポーランド記法(RPN) の電卓を実装せよという問題を解いた時には、この電卓は文法的に正しい入力が与えられる限り正しく動くよ、という注意書きがありました

In [13]:
def foldingFunction(list: List[Double], next: String): List[Double] = (list, next) match {
    case (x :: y :: ys, "*") => (y * x) :: ys
    case (x :: y :: ys, "+") => (y + x) :: ys
    case (x :: y :: ys, "-") => (y - x) :: ys
    case (xs, numString) => numString.toInt :: xs
}

defined function foldingFunction

In [15]:
def solveRPN(s: String): Double = (s.split(' ').toList.foldLeft(Nil: List[Double]) {foldingFunction}).head

defined function solveRPN

In [16]:
solveRPN("10 4 3 + 2 * -")

res15: Double = -4.0

In [17]:
"1".parseInt.toOption

res16: Option[Int] = Some(1)

In [18]:
"foo".parseInt.toOption

res17: Option[Int] = None

In [19]:
def foldingFunction(list: List[Double], next: String): Option[List[Double]] =  (list, next) match {
    case (x :: y :: ys, "*") => ((y * x) :: ys).point[Option]
    case (x :: y :: ys, "+") => ((y + x) :: ys).point[Option]
    case (x :: y :: ys, "-") => ((y - x) :: ys).point[Option]
    case (xs, numString) => numString.parseInt.toOption map {_ :: xs}
}

defined function foldingFunction

In [20]:
foldingFunction(List(3, 2), "*")

res19: Option[List[Double]] = Some(List(6.0))

In [21]:
foldingFunction(Nil, "*")

res20: Option[List[Double]] = None

In [22]:
foldingFunction(Nil, "wawa")

res21: Option[List[Double]] = None

In [23]:
def solveRPN(s: String): Option[Double] = for {
    List(x) <- s.split(' ').toList.foldLeftM(Nil: List[Double]) {foldingFunction}
} yield x

defined function solveRPN

In [24]:
solveRPN("1 2 * 4 +")

res23: Option[Double] = Some(6.0)

In [25]:
solveRPN("1 2 * 4")

res24: Option[Double] = None

In [26]:
solveRPN("1 8 garbage")

res25: Option[Double] = None

### モナディック関数の合成
* 第１３章でモナド則を紹介した時、 <=< 関数は関数合成によく似ているけど、普通の a -> b ではなくて、 a -> m b みたいなモナディック関数に作用するのだと言いました

### Kleisli
* Scalaz には Kleisli と呼ばれる A => M[B] という方の関数に対する特殊なラッパーがある

```scala
sealed trait Kleisli[M[+_], -A, +B] { self =>
  def run(a: A): M[B]
  ...
  def >=>[C](k: Kleisli[M, B, C])(implicit b: Bind[M]): Kleisli[M, A, C] = kleisli((a: A) => b.bind(this(a))(k(_)))
  def addThen[C](k: Kleisli[M, B, C])(implicit b: Bind[M]): Kleisli[M, A, C] = this >=> k
  def <=<[C](k: Kleisli[M, C, A])(implicit b: Bind[M]): Kleisli[M, C, B] = k >=> this
  def compose[C](k: Kleisli[M, C, A])(implicit b: Bind[M]): Kleisli[M, C, B] = k >=> this
  ...
}

object Kleisli extends KleisliFunctions with KleisliInstances {
    def apply[M[+_], A, B](f: A => M[B]): Kleisli[M, A, B] = kleisli(f)
}
```

In [27]:
val f = Kleisli { (x: Int) => (x + 1).some }

f: Kleisli[Option, Int, Int] = Kleisli(<function1>)

In [28]:
val g = Kleisli { (x: Int) => (x * 100).some }

g: Kleisli[Option, Int, Int] = Kleisli(<function1>)

In [29]:
4.some >>= (f <=< g)

res28: Option[Int] = Some(401)

In [30]:
4.some >>= (f >=> g)

res29: Option[Int] = Some(500)

### Reader 再び
* Scalaz は Reader を Kleisli の特殊系として以下のように定義する

```scala
type ReaderT[F[+_], E, A] = Kleisli[F, E, A]
type Reader[E, A] = ReaderT[Id, E, A]
object Reader {
    def apply[E, A](f: E => A): Reader[E, A] = Kleisli[Id, E, A](f)
}
```

In [31]:
val addStuff: Reader[Int, Int] = for {
    a <- Reader { (_: Int) * 2 }
    b <- Reader { (_: Int) + 10 }
} yield a + b

addStuff: Reader[Int, Int] = Kleisli(<function1>)

In [32]:
addStuff(3)

res31: Id[Int] = 19

### モナドを作る
* この説では、方が生まれてモナドであると確認され、適切な Monad インスタンスが与えられるまでの過程を、例題を通して学ぼうと思います。
* ...[3, 5, 9] のような非決定的値を表現したいのだけど、さらに 3 である確率は 50 パーセント、5と9である確率はそれぞれ25パーセントである、ということを表したくなったらどうしましょう?

In [33]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [34]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    implicit val probInstance = new Functor[Prob] {
        def map[A, B](fa: Prob[A])(f: A => B): Prob[B] =
        Prob(fa.list map { case (x, p) => (f(x), p) })
    }
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [35]:
Prob((3, 0.5) :: (5, 0.25) :: (9, 0.25) :: Nil) map {-_}

res34: Prob[Int] = Prob(List((-3, 0.5), (-5, 0.25), (-9, 0.25)))

In [36]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    def flatten[B](xs: Prob[Prob[B]]): Prob[B] = {
        def multall(innerxs: Prob[B], p: Double) =
        innerxs.list map { case (x, r) => (x, p * r) }
        Prob((xs.list map { case (innerxs, p) => multall(innerxs, p) }).flatten)
    }
    
    implicit val probInstance = new Functor[Prob] {
        def map[A, B](fa: Prob[A])(f: A => B): Prob[B] =
        Prob(fa.list map { case (x, p) => (f(x), p) })
    }
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [38]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    def flatten[B](xs: Prob[Prob[B]]): Prob[B] = {
        def multall(innerxs: Prob[B], p: Double) =
        innerxs.list map { case (x, r) => (x, p * r) }
        Prob((xs.list map { case (innerxs, p) => multall(innerxs, p) }).flatten)
    }
    
    implicit val probInstance = new Functor[Prob] with Monad[Prob] {
        def point[A](a: => A): Prob[A] = Prob((a, 1.0) :: Nil)
        def bind[A, B](fa: Prob[A])(f: A => Prob[B]): Prob[B] = flatten(map(fa)(f))
        override def map[A, B](fa: Prob[A])(f: A => B): Prob[B] =
        Prob(fa.list map { case (x, p) => (f(x), p) })
    }
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [44]:
sealed trait Coin
object Coin {
    case object Heads extends Coin
    case object Tails extends Coin
    implicit val coinEqual: Equal[Coin] = Equal.equalA
    
    def coin: Prob[Coin] = Prob(Heads -> 0.5 :: Tails -> 0.5 :: Nil)
    def loadedCoin: Prob[Coin] = Prob(Heads -> 0.1 :: Tails -> 0.9 :: Nil)
    
    def flipThree: Prob[Boolean] = for {
        a <- coin
        b <- coin
        c <- loadedCoin
    } yield { List(a, b, c) all {_ === Tails} }
}

defined trait Coin
defined object Coin

In [45]:
Coin.flipThree

res44: Prob[Boolean] = Prob(
  List(
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (true, 0.225)
  )
)